In [115]:
import os

interface_dir = os.environ["DATA"] + "webinterfaces/int01_prototype/"

output_json_dir = "/home/jleguy/Documents/postdoc/git_repos/WebXAII/functions/data/"
output_assets_dir = "/home/jleguy/Documents/postdoc/git_repos/WebXAII/src/assets/"

users = {
    "H" : ["H" + str(i) for i in range(1, 11)],
    "AI": ["AI" + str(i) for i in range(1, 11)],
    "AI+XAI_SHAP": ["AI_SHAP" + str(i) for i in range(1, 11)],
    "AI+XAI_CF": ["AI_CF" + str(i) for i in range(1, 11)],
    "AI+XAI_LLM": ["AI_LLM" + str(i) for i in range(1, 11)]
}


In [116]:
CONDITIONS = ["H", "AI", "AI+XAI_SHAP", "AI+XAI_CF", "AI+XAI_LLM"]

TRAINING_TASKS = ["task00", "task00"]
EASY_TASKS = ["task01", "task01"]
DIFFICULT_TASKS = ["task02", "task02"]
TIME_MILD = 20
TIME_STRONG = 10

JSON_PATH_CONSENT_FORM = os.environ["DATA"] + "webinterfaces/int01_prototype/res/interface_views/000_consent_form.json"
JSON_PATH_INTRO_FORM = os.environ["DATA"] + "webinterfaces/int01_prototype/res/interface_views/010_introduction.json"
JSON_PATH_INSTRUCTIONS_FORM = os.environ["DATA"] + "webinterfaces/int01_prototype/res/interface_views/020_instructions.json"
JSON_PATH_TRAINING_1_INTRO = os.environ["DATA"] + "webinterfaces/int01_prototype/res/interface_views/030_training_1_introduction.json"
JSON_PATH_TRAINING_2_INTRO = os.environ["DATA"] + "webinterfaces/int01_prototype/res/interface_views/032_training_2_introduction.json"
JSON_PATH_SESSION_1_INTRO = os.environ["DATA"] + "webinterfaces/int01_prototype/res/interface_views/040_session_1_introduction.json"
JSON_PATH_SESSION_2_INTRO = os.environ["DATA"] + "webinterfaces/int01_prototype/res/interface_views/042_session_2_introduction.json"
JSON_PATH_SESSION_3_INTRO = os.environ["DATA"] + "webinterfaces/int01_prototype/res/interface_views/044_session_3_introduction.json"
JSON_PATH_SESSION_4_INTRO = os.environ["DATA"] + "webinterfaces/int01_prototype/res/interface_views/046_session_4_introduction.json"
JSON_PATH_TASK_END_COGLOAD = os.environ["DATA"] + "webinterfaces/int01_prototype/res/interface_views/050_task_end_quest_cogload.json"
JSON_PATH_TASK_END_QUEST_AI = os.environ["DATA"] + "webinterfaces/int01_prototype/res/interface_views/060_task_end_quest_AI.json"
JSON_PATH_TASK_END_QUEST_XAI = os.environ["DATA"] + "webinterfaces/int01_prototype/res/interface_views/070_task_end_quest_XAI.json"
JSON_PATH_FINAL_QUEST_INTRO = os.environ["DATA"] + "webinterfaces/int01_prototype/res/interface_views/080_final_quest_introduction.json"
JSON_PATH_FINAL_QUEST_DEMOGRAPHICS = os.environ["DATA"] + "webinterfaces/int01_prototype/res/interface_views/090_final_quest_demographics.json"
JSON_PATH_FINAL_QUEST_NFC = os.environ["DATA"] + "webinterfaces/int01_prototype/res/interface_views/100_final_quest_nfc.json"
JSON_PATH_END = os.environ["DATA"] + "webinterfaces/int01_prototype/res/interface_views/110_end.json"

In [117]:
import csv

def load_json(json_path):
    with open(json_path) as json_data:
        d = json.load(json_data)
        json_data.close()
        return d

def load_task_csv(csv_path):
    return list(csv.DictReader(open(csv_path), delimiter=','))


In [118]:
import csv


def gen_page_new_rule(rule_text, training=False):

    return {
        "type": "p-instruction",
        "title": "New " + ("training " if training else "") + "task",
        "body_text": "The rule is : " + rule_text,
        "with_button": True,
        "button_text": "Start the " + ("training " if training else "") + "task",
    }

def get_column_csv_xai(condition):
    if condition == "AI+XAI_SHAP":
        return "xai_shap"
    elif condition == "AI+XAI_CF":
        return "xai_cf"
    elif condition == "AI+XAI_LLM":
        return "xai_llm"
    else:
        return ""

def generate_task_instances(rule_csv_path, condition):

    output_instances_d = []

    csv_content = load_task_csv(rule_csv_path)

    for idx in range(len(csv_content)):

        model_d = {
            "is_image": False,
            "label": "The AI prediction is : " + ("YES" if csv_content[idx]["pred"] == "1" else "NO"),
            "title": "AI prediction"
        } if "ai" in condition.lower() else {}

        explanation_d = []
        if "xai" in condition.lower():
            label_explanation = csv_content[idx][get_column_csv_xai(condition)]
            if not "llm" in condition.lower():
                label_explanation = os.path.join("assets", label_explanation)

            explanation_d = [
                {
                  "is_image": not "llm" in condition.lower(),
                  "label": label_explanation,
                  "title": "Explanation of the AI's prediction"
                }
              ]

        input_d = {
            "is_image": True,
            "label": os.path.join("assets", csv_content[idx]["source"]),
            "title": "Image to analyze"
        }

        output_instances_d.append(
            {
                "model": model_d,
                "explanation": explanation_d,
                "input": input_d,
                "expected" : 0 if csv_content[idx]["target"] == "1" else 1,
            }
        )

    return output_instances_d

def generate_task_page(rule_txt, rule_csv_path, condition, time_limit, activate_feedback_answer=False):

    return {
      "type": "p-task",
      "title": "Identify if the image respects the rule before the end of the timer.",
      "desc": "Rule : " + rule_txt,
      "show_progression_bar": False,
      "randomize": True,
      "timer": time_limit,
      "feedback_answer_activated": activate_feedback_answer,
      "feedback_answer_correct": "This was the right answer",
      "feedback_answer_wrong": "This was not the right answer",
      "feedback_answer_show_expected": False,
      "feedback_answer_expected_text": "Expected answer : ",
      "instances": generate_task_instances(rule_csv_path, condition),
      "question": {
        "type": "radio",
        "primary_text": "",
        "secondary_text": "Does the image presented respect the rule ?",
        "answers": [
          "Yes",
          "No"
        ]
      }
    }

def generate_page_score():
    return {
        "type": 'p-instruction',
        "title": "Task end",
        "body_text" : "On the last task, you achieved a score of",
        "with_button": True,
        "button_text": 'Next',
        "score" : True
    }

In [119]:
def generate_instructions_page(title, content, button_text=None):
    return {
      "type": "p-instruction",
      "title": title,
      "body_text": content,
      "button_text": button_text if button_text else "Next",
      "with_button": True
    }

In [120]:
def generate_task(task, condition, time, is_training):

    rule_text = load_json(os.path.join(interface_dir, "res", "tasks", task+".json"))["rule"]
    rule_csv_path = os.path.join(interface_dir, "res", "tasks", task+"_content.csv")

    task_json_list = []
    task_json_list.append(gen_page_new_rule(rule_text, condition))
    task_json_list.append(generate_task_page(rule_text, rule_csv_path, condition, time, activate_feedback_answer=is_training))
    task_json_list.append(generate_page_score())
    return task_json_list

def generate_end_of_session_questionnaire():
    pass

def generate_session(condition, tasks, time_limit):

    session_json_list = []
    for task in tasks:
        session_json_list.append(generate_task(task, condition, time_limit, is_training=False))
    session_json_list.append(generate_end_of_session_questionnaire())

def create_protocol_json(condition, training_tasks, easy_tasks, difficult_tasks, time_mild, time_strong):
    """
    :param condition: Either "H", "AI", "AI+XAI_SHAP", "AI+XAI_CF", "AI+XAI_LLM"
    :return:
    """
    full_list = []

    full_list.append(load_json(JSON_PATH_TRAINING_1_INTRO))
    full_list.extend(generate_task(training_tasks[0], condition, -1, True))
    full_list.append(load_json(JSON_PATH_TRAINING_2_INTRO))
    full_list.extend(generate_task(training_tasks[1], condition, time_mild, True))

    full_list.append(load_json(JSON_PATH_SESSION_1_INTRO))
    full_list.extend(generate_task(easy_tasks[0:2], condition, time_mild, True))

    full_list.append(load_json(JSON_PATH_FINAL_QUEST_NFC))

    return full_list


In [121]:
import shutil
import json

def create_users_json(conditions, users):
    output_dict = {}

    for condition in conditions:
        for user in users[condition]:
            output_dict[user] = {"roles": "user", "protocol": condition+".json"}

    return output_dict

def deploy(output_json_dir, output_assets_dir, users):

    protocols_path = os.path.join(output_json_dir, "protocols")
    users_path = os.path.join(output_json_dir, "users.json")
    assets_output_path = os.path.join(output_assets_dir, "res")

    # Removing former installation
    shutil.rmtree(protocols_path, ignore_errors=True)
    try:
        os.remove(users_path)
    except FileNotFoundError:
        pass

    os.makedirs(protocols_path)

    for condition in CONDITIONS:
        # Writing of JSON protocol files
        protocol_json = create_protocol_json(condition, TRAINING_TASKS, EASY_TASKS, DIFFICULT_TASKS, TIME_MILD, TIME_STRONG)
        with open(os.path.join(protocols_path, condition+".json"), 'w') as f:
            json.dump(protocol_json, f, indent=4)

    # Writing users json file
    users_json = create_users_json(CONDITIONS, users)
    with open(users_path, 'w') as f:
        json.dump(users_json, f, indent=4)

    # Copy assets
    shutil.rmtree(assets_output_path, ignore_errors=True)
    shutil.copytree(os.path.join(interface_dir, "res"), assets_output_path)


In [122]:
deploy(output_json_dir, output_assets_dir, users)

TypeError: can only concatenate list (not "str") to list